In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as multi
from tqdm import tqdm

In [7]:
metadata_dict = {}
metadata = pd.read_csv('/home/mar/Documents/TFM/GitHub/HB_PublicData/1-Obtaining-DEGs-for-HB/Metadata_HB/HB_joint_METADATA.tsv',sep='\t')
lst = []
for i in metadata['type']:
    if str(i).startswith('Hepatoblastoma'):
        lst.append('case')
    elif str(i).startswith('Normal'):
        lst.append('control')
    else:
        lst.append('other')
metadata['class'] = lst
metadata_dict = pd.Series(metadata['class'].values,index=metadata['sample']).to_dict()


In [9]:
Carrillo_path = '/home/mar/Documents/TFM/GitHub/HB_PublicData/1-Obtaining-DEGs-for-HB/Matrices_HB/Originals_HB/Carrillo-Reixach_RNAseq_log2.xlsx'
data = pd.read_excel(Carrillo_path, index_col=0)
data.head()

,HB10NT,HB11NT,HB13NT,HB14NT,HB1NT,HB16NT,HB17NT,HB45NT,HB76NT,HB77NT,...,MABHB73T,MABHB80T,MABHB225T,MABHB238T,MABHB233T,MABHB235T,HB5T,MABHB236T,MABHB239T,MABHB252T
Gene,,,,,,,,,,,,,,,,,,,,,
AMDHD1,5.877091,7.434907,8.140415,7.162385,7.512613,8.130875,6.521973,6.254472,6.259856,7.117445,...,5.648395,4.991604,5.282936,5.593223,3.126332,3.877451,4.100993,5.511044,3.856974,6.928588
ANGPTL6,4.266450,4.109980,4.480419,4.206414,3.906074,4.680423,4.037238,3.363358,3.772616,4.568131,...,0.450849,-0.273450,-1.562796,-2.595718,-1.335339,-1.780055,-1.089576,-3.862953,-2.914041,-3.250376
CCDC50,6.654351,6.522819,6.729777,6.814082,6.876634,6.662568,7.103469,8.185944,7.793411,7.418240,...,6.680109,6.129098,6.067138,5.720340,6.748458,6.004517,6.674247,6.501997,5.378468,6.377879
CFI,9.127665,9.463506,10.283812,9.639721,9.894176,9.387165,9.249945,9.842175,9.354646,9.899805,...,7.453105,8.664987,8.928759,4.364616,7.928600,7.605187,7.503656,6.040356,5.635353,8.099088
CGREF1,0.969393,1.700824,1.280934,1.037333,1.115303,2.482406,1.481028,2.403946,4.351898,2.510797,...,5.187645,3.913090,2.956020,6.436723,4.445699,5.134513,4.377009,4.223685,3.774045,3.726993


In [8]:
data.head()
data.columns = data.columns.map(metadata_dict)
data = data.rename_axis('gene')

,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gene,,,,,,,,,,,,,,,,,,,,,
AMDHD1,5.877091,7.434907,8.140415,7.162385,7.512613,8.130875,6.521973,6.254472,6.259856,7.117445,...,5.648395,4.991604,5.282936,5.593223,3.126332,3.877451,4.100993,5.511044,3.856974,6.928588
ANGPTL6,4.266450,4.109980,4.480419,4.206414,3.906074,4.680423,4.037238,3.363358,3.772616,4.568131,...,0.450849,-0.273450,-1.562796,-2.595718,-1.335339,-1.780055,-1.089576,-3.862953,-2.914041,-3.250376
CCDC50,6.654351,6.522819,6.729777,6.814082,6.876634,6.662568,7.103469,8.185944,7.793411,7.418240,...,6.680109,6.129098,6.067138,5.720340,6.748458,6.004517,6.674247,6.501997,5.378468,6.377879
CFI,9.127665,9.463506,10.283812,9.639721,9.894176,9.387165,9.249945,9.842175,9.354646,9.899805,...,7.453105,8.664987,8.928759,4.364616,7.928600,7.605187,7.503656,6.040356,5.635353,8.099088
CGREF1,0.969393,1.700824,1.280934,1.037333,1.115303,2.482406,1.481028,2.403946,4.351898,2.510797,...,5.187645,3.913090,2.956020,6.436723,4.445699,5.134513,4.377009,4.223685,3.774045,3.726993
